In [3]:
# !pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [125]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [41]:
#Testing pyspark installation
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark-3.3.1-bin-hadoop3'

In [42]:
#Initiate Spark Context
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local[*]')
sc = pyspark.SparkContext(conf=conf)
spark  = SparkSession(sc)

In [43]:
spark

In [100]:
#Stop the Session
sc.stop()

In [22]:
from impala.dbapi import connect
conn = connect(host='192.168.118.137', port=21050)
cursor = conn.cursor()
cursor.execute('select * from vel_riskfactor')
# print(cursor)  # prints the result set's schema
results = cursor.fetchall()
for data in results:
        print(data)

('unsafe following distance', 54.0, 6.323071002960205)
('lane departure', 45.0, 3.116119146347046)
('overspeed', 86.0, 4.442246437072754)
('overspeed', 89.5, 7.700280666351318)
('overspeed', 85.0, 10.987820625305176)
('overspeed', 82.5, 6.337808609008789)
('unsafe tail distance', 27.0, 7.455998420715332)
('unsafe tail distance', 40.0, 9.151140213012695)
('unsafe following distance', 68.0, 2.92341685295105)
('unsafe tail distance', 63.0, 5.985864162445068)
('lane departure', 48.0, 1.5608904361724854)
('unsafe tail distance', 54.0, 7.513645172119141)
('unsafe following distance', 54.0, 4.575402736663818)
('overspeed', 90.0, 7.748658180236816)
('unsafe following distance', 48.0, 4.648604393005371)
('overspeed', 85.66666666666667, 7.673183917999268)
('lane departure', 24.0, 8.92822551727295)
('unsafe tail distance', 53.0, 3.064993381500244)
('unsafe tail distance', 48.66666666666666, 30.101442337036133)
('overspeed', 88.0, 4.7015604972839355)
('overspeed', 86.0, 6.118200302124023)
('oversp

In [5]:
df = spark.createDataFrame(results)

In [6]:
df.head()

Row(_1='driverid', _2='truckid', _3='model', _4=None, _5=None, _6=None, _7=None, _8=None, _9=None, _10=None, _11=None, _12=None, _13=None, _14=None, _15=None, _16=None, _17=None, _18=None, _19=None, _20=None, _21=None, _22=None, _23=None, _24=None, _25=None, _26=None, _27=None, _28=None, _29=None, _30=None, _31=None, _32=None, _33=None, _34=None, _35=None, _36=None, _37=None, _38=None, _39=None, _40=None, _41=None, _42=None, _43=None, _44=None, _45=None, _46=None, _47=None, _48=None, _49=None, _50=None, _51=None, _52=None, _53=None, _54=None, _55=None, _56=None, _57=None, _58=None, _59=None, _60=None, _61=None, _62=None, _63=None, _64=None, _65=None, _66=None, _67=None, _68=None, _69=None, _70=None, _71=None, _72=None, _73=None, _74=None, _75=None, _76=None, _77=None, _78=None, _79=None, _80=None, _81=None, _82=None, _83=None, _84=None, _85=None, _86=None, _87=None, _88=None, _89=None, _90=None, _91=None, _92=None, _93=None, _94=None, _95=None, _96=None, _97=None, _98=None, _99=None, _

In [83]:
from impala.util import as_pandas

cursor.execute('SELECT * FROM riskfactor')
riskfactor = as_pandas(cursor)

In [84]:
riskfactor.head()

,driverid,events,totmiles,riskfactor
0,A1,3,628507,4.773216
1,A2,1,664543,1.504794
2,A3,6,639584,9.381098
3,A4,6,663289,9.045831
4,A5,8,676574,11.824280


In [37]:
from impala.util import as_pandas

cursor.execute('SELECT * FROM vel_mod_riskfactor')
vel_mod_riskfactor = as_pandas(cursor)

In [38]:
vel_mod_riskfactor.head()

,event,avg_velocity,totmiles,model,riskfactor
0,unsafe following distance,54.0,632604.0,Navistar,6.323071
1,lane departure,45.0,641824.0,Kenworth,3.116119
2,overspeed,86.0,675334.0,Navistar,4.442246
3,overspeed,89.5,649327.0,Hino,7.700281
4,overspeed,85.0,637069.0,Ford,10.987821


In [39]:
vel_mod_riskfactor.to_csv('vel_mod_riskfactor.csv')

In [87]:
# df = avg_velocity.merge(riskfactor,how='left',on='driverid')

In [88]:
# df =df[['event','avg_velocity','riskfactor']]

In [44]:
dataset = spark.read.csv("vel_mod_riskfactor.csv", inferSchema=True, header =True)
dataset.printSchema()

root
 |-- event: string (nullable = true)
 |-- avg_velocity: double (nullable = true)
 |-- totmiles: integer (nullable = true)
 |-- model: string (nullable = true)
 |-- riskfactor: double (nullable = true)



In [48]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="event", outputCol="event_index")
indexed = indexer.fit(dataset).transform(dataset)
indexed.show()

+--------------------+------------+--------+------------+-----------+-----------+
|               event|avg_velocity|totmiles|       model| riskfactor|event_index|
+--------------------+------------+--------+------------+-----------+-----------+
|unsafe following ...|        54.0|  632604|    Navistar|6.323071003|        1.0|
|      lane departure|        45.0|  641824|    Kenworth|3.116119146|        0.0|
|           overspeed|        86.0|  675334|    Navistar|4.442246437|        2.0|
|           overspeed|        89.5|  649327|        Hino|7.700280666|        2.0|
|           overspeed|        85.0|  637069|        Ford|10.98782063|        2.0|
|           overspeed|        82.5|  631133|   Peterbilt|6.337808609|        2.0|
|unsafe tail distance|        27.0|  670601|   Peterbilt|7.455998421|        3.0|
|unsafe tail distance|        40.0|  655656|Freightliner|9.151140213|        3.0|
|unsafe following ...|        68.0|  684131|        Ford|2.923416853|        1.0|
|unsafe tail dis

In [49]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="model", outputCol="model_index")
indexed = indexer.fit(indexed).transform(indexed)
indexed.show()

+--------------------+------------+--------+------------+-----------+-----------+-----------+
|               event|avg_velocity|totmiles|       model| riskfactor|event_index|model_index|
+--------------------+------------+--------+------------+-----------+-----------+-----------+
|unsafe following ...|        54.0|  632604|    Navistar|6.323071003|        1.0|        5.0|
|      lane departure|        45.0|  641824|    Kenworth|3.116119146|        0.0|        7.0|
|           overspeed|        86.0|  675334|    Navistar|4.442246437|        2.0|        5.0|
|           overspeed|        89.5|  649327|        Hino|7.700280666|        2.0|        4.0|
|           overspeed|        85.0|  637069|        Ford|10.98782063|        2.0|        0.0|
|           overspeed|        82.5|  631133|   Peterbilt|6.337808609|        2.0|        1.0|
|unsafe tail distance|        27.0|  670601|   Peterbilt|7.455998421|        3.0|        1.0|
|unsafe tail distance|        40.0|  655656|Freightliner|9.1

In [50]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['event_index','model_index','avg_velocity','totmiles'], outputCol = "Independent Features")
rf_df = vectorAssembler.transform(indexed)
rf_df = rf_df.select(['Independent Features', 'riskfactor'])
rf_df.show(5)

+--------------------+-----------+
|Independent Features| riskfactor|
+--------------------+-----------+
|[1.0,5.0,54.0,632...|6.323071003|
|[0.0,7.0,45.0,641...|3.116119146|
|[2.0,5.0,86.0,675...|4.442246437|
|[2.0,4.0,89.5,649...|7.700280666|
|[2.0,0.0,85.0,637...|10.98782063|
+--------------------+-----------+
only showing top 5 rows



In [51]:
splits = rf_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [52]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'Independent Features', labelCol='riskfactor')
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.2841821164764467,0.006445818527117019,0.0003232271538839428,-3.255382169839224e-07]
Intercept: 6.741412535494262


In [53]:
lr_model.summary

In [54]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 3.539470
r2: 0.007665


In [96]:
lr_predictions = lr_model.transform(test_df)
lr_evaluator = RegressionEvaluator(labelCol="riskfactor", predictionCol="prediction", metricName="rmse")
rmse = lr_evaluator.evaluate(lr_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 4.50045


In [97]:
print(lr_evaluator.evaluate(lr_predictions,
{lr_evaluator.metricName: "r2"}))
print(lr_evaluator.evaluate(lr_predictions,
{lr_evaluator.metricName: "mse"}))
print(lr_evaluator.evaluate(lr_predictions,
{lr_evaluator.metricName: "rmse"}))
print(lr_evaluator.evaluate(lr_predictions,
{lr_evaluator.metricName: "mae"}))

-0.011620156724656994
20.254028111169536
4.5004475456524915
2.718979113254493


In [80]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
dt = DecisionTreeRegressor(featuresCol = 'Independent Features', labelCol='riskfactor')
dt_model = dt.fit(train_df)

In [81]:
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(labelCol="riskfactor", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 2.63397


In [82]:
dt_model.featureImportances

SparseVector(4, {0: 0.0167, 1: 0.4534, 2: 0.0313, 3: 0.4986})

In [83]:
print(dt_model)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_619897527b17, depth=5, numNodes=57, numFeatures=4


In [85]:
print(dt_evaluator.evaluate(dt_predictions,
{dt_evaluator.metricName: "r2"}))
print(dt_evaluator.evaluate(dt_predictions,
{dt_evaluator.metricName: "mse"}))
print(dt_evaluator.evaluate(dt_predictions,
{dt_evaluator.metricName: "rmse"}))
print(dt_evaluator.evaluate(dt_predictions,
{dt_evaluator.metricName: "mae"}))

0.6534800158911024
6.93780709347205
2.6339717336129578
1.9398624760258603


In [86]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'Independent Features', labelCol='riskfactor', maxIter=50)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'riskfactor', 'Independent Features').show(5)

+------------------+-----------+--------------------+
|        prediction| riskfactor|Independent Features|
+------------------+-----------+--------------------+
| 9.281641224894914|1.504793525|[0.0,0.0,24.0,664...|
|5.2825605314261015|4.620260715|[0.0,0.0,42.33333...|
|10.108126824450343|6.253292561|[0.0,0.0,45.0,639...|
| 6.439059685518736|7.680067062|[0.0,0.0,51.5,651...|
| 9.952370451495787|7.862290859|[0.0,0.0,55.0,635...|
+------------------+-----------+--------------------+
only showing top 5 rows



In [88]:
gbt_evaluator = RegressionEvaluator(
    labelCol="riskfactor", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 2.37893


In [89]:
print(gbt_evaluator.evaluate(gbt_predictions,
{gbt_evaluator.metricName: "r2"}))
print(gbt_evaluator.evaluate(gbt_predictions,
{gbt_evaluator.metricName: "mse"}))
print(gbt_evaluator.evaluate(gbt_predictions,
{gbt_evaluator.metricName: "rmse"}))
print(gbt_evaluator.evaluate(gbt_predictions,
{gbt_evaluator.metricName: "mae"}))

0.7173370090321838
5.659302186687903
2.3789287897471634
1.852273894969654


In [99]:
gbt_model.featureImportances

SparseVector(4, {0: 0.0406, 1: 0.2997, 2: 0.2492, 3: 0.4104})

In [90]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
rf = RandomForestRegressor(featuresCol = 'Independent Features', labelCol='riskfactor')
rf_model = rf.fit(train_df)

In [91]:
rf_predictions = rf_model.transform(test_df)
rf_evaluator = RegressionEvaluator(labelCol="riskfactor", predictionCol="prediction", metricName="rmse")
rmse = rf_evaluator.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 3.65276


In [92]:
rf_model.featureImportances

SparseVector(4, {0: 0.062, 1: 0.3474, 2: 0.1824, 3: 0.4083})

In [93]:
print(rf_model)

RandomForestRegressionModel: uid=RandomForestRegressor_0c2866a9f576, numTrees=20, numFeatures=4


In [94]:
print(rf_evaluator.evaluate(rf_predictions,
{rf_evaluator.metricName: "r2"}))
print(rf_evaluator.evaluate(rf_predictions,
{rf_evaluator.metricName: "mse"}))
print(rf_evaluator.evaluate(rf_predictions,
{rf_evaluator.metricName: "rmse"}))
print(rf_evaluator.evaluate(rf_predictions,
{rf_evaluator.metricName: "mae"}))

0.33358059913924876
13.342633783182134
3.652757011242622
2.7128170195315024


In [14]:
import pyforest

In [18]:
from pyspark.ml.regression import LinearRegression


lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

lrModel = lr.fit(riskfactor[['events','totmiles']])

AttributeError: 'DataFrame' object has no attribute '_jdf'

In [6]:
geolocation.head()

,truckid,driverid,event,latitude,longitude,city,state,velocity,event_ind,idling_ind
0,truckid,driverid,event,NaN,NaN,city,state,NaN,NaN,NaN
1,A54,A54,normal,38.440467,-122.714431,Santa Rosa,California,17.0,0.0,0.0
2,A20,A20,normal,36.977173,-121.899402,Aptos,California,27.0,0.0,0.0
3,A40,A40,overspeed,37.957702,-121.290780,Stockton,California,77.0,1.0,0.0
4,A31,A31,normal,39.409608,-123.355566,Willits,California,22.0,0.0,0.0


In [8]:
drivermileage.head()

,driverid,totmiles
0,A1,628507
1,A10,675377
2,A100,634338
3,A11,652452
4,A12,668241


In [9]:
geolocation.query("event != 'normal' & event_ind == 1 & idling_ind == 0")

,truckid,driverid,event,latitude,longitude,city,state,velocity,event_ind,idling_ind
3,A40,A40,overspeed,37.957702,-121.290780,Stockton,California,77.0,1.0,0.0
13,A5,A5,unsafe following distance,41.526509,-124.038407,Klamath,California,33.0,1.0,0.0
15,A48,A48,overspeed,38.752124,-121.288006,Roseville,California,77.0,1.0,0.0
25,A49,A49,lane departure,39.150171,-123.207783,Ukiah,California,41.0,1.0,0.0
86,A97,A97,overspeed,38.440467,-122.714431,Santa Rosa,California,33.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7877,A27,A27,unsafe following distance,39.409608,-123.355566,Willits,California,54.0,1.0,0.0
7895,A90,A90,unsafe following distance,39.017396,-122.057748,Arbuckle,California,31.0,1.0,0.0
7943,A84,A84,overspeed,33.663357,-116.310009,La Quinta,California,81.0,1.0,0.0
7977,A97,A97,unsafe tail distance,36.977173,-121.899402,Aptos,California,66.0,1.0,0.0
